In [3]:
import requests

def get_full_playlist(playlist_id):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Referer": "https://music.163.com/"
    }

    # 1. 获取歌单详情
    detail_url = f"https://music.163.com/api/v1/playlist/detail?id={playlist_id}"
    try:
        res = requests.get(detail_url, headers=headers, timeout=10)
        data = res.json()
    except Exception as e:
        print(f"网络请求失败: {e}")
        return

    if data.get('code') != 200:
        print("获取失败，请检查歌单 ID 是否正确且已设为公开。")
        return

    # 提取所有歌曲 ID
    track_ids = [str(item['id']) for item in data['playlist']['trackIds']]
    total_count = len(track_ids)
    print(f"歌单名称: {data['playlist']['name']}")
    print(f"检测到总计 {total_count} 首歌曲，正在分批抓取详情并清洗数据...")

    # 2. 分批次获取详情（每批 50 首）
    all_tracks = []
    for i in range(0, total_count, 50):
        batch_ids = track_ids[i:i+50]
        # 注意：这里改用更稳定的 song/detail 接口格式
        song_url = f"https://music.163.com/api/song/detail?ids=[{','.join(batch_ids)}]"
        try:
            song_res = requests.get(song_url, headers=headers, timeout=10)
            songs_data = song_res.json()
            all_tracks.extend(songs_data.get('songs', []))
        except:
            print(f"抓取第 {i+1} 批次时出错，已跳过。")

    # 3. 导出结果（增加防御性逻辑）
    filename = f"歌单_{playlist_id}_197首全量导出.txt"
    with open(filename, "w", encoding="utf-8") as f:
        for idx, track in enumerate(all_tracks, 1):
            # 兼容性处理：如果 'name' 或 'ar' 不存在，不至于报错
            name = track.get('name', '未知歌曲')
            
            # 重点：同时检查 'ar' 和 'artists' 两个可能的字段
            artist_data = track.get('ar') or track.get('artists') or []
            artist_names = [a.get('name', '未知歌手') for a in artist_data]
            artists = "/".join(artist_names) if artist_names else "未知歌手"
            
            line = f"{idx}. {name} - {artists}"
            f.write(line + "\n")
            
            # 控制台只打印前 5 首做预览
            if idx <= 5: 
                print(f"已捕获: {line}")

    print(f"\n✅ 导出完成！")
    print(f"共计成功解析: {len(all_tracks)} 首歌曲")
    print(f"文件保存在: {filename}")

# 执行
get_full_playlist("17382710871")

歌单名称: dyxj
检测到总计 197 首歌曲，正在分批抓取详情并清洗数据...
已捕获: 1. 南山雪 - 小皇堡（LKB）
已捕获: 2. 风催雨 (弹唱版) - 费戚戚
已捕获: 3. 最初的记忆（DJ热播版） - 奇衡三
已捕获: 4. 彩月​-​Sai​-​getsu- - Otokaze
已捕获: 5. 探故知 - 吉泽树

✅ 导出完成！
共计成功解析: 197 首歌曲
文件保存在: 歌单_17382710871_197首全量导出.txt
